## ML-Assignment 2
Group Members:

Muhammed Talha Gedikli - 19069608

Ahmet Emin Demirbaş - 19069927

Erva Güney - 20069046

Ökkeş Kaan Kırmacı - 18069013

## Import Libraries and Load Data

In [1]:
# Load important libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, cross_val_score

# Import data
train_data = pd.read_excel("./asg1_train_test_data/asg1_traindata.xlsx")
test_data = pd.read_excel("./asg1_train_test_data/asg1_testdata_without_prices.xlsx")


# Split features and prices
features_X = ["Year", "Type", "Shift", "km", "Power", "Engine", "Seller"]
features_y = ["Price"]


# We use 'without_prices' to refer to 'asg1_testdata_without_prices.xlsx'
X_data_without_prices = pd.read_excel("./asg1_train_test_data/asg1_testdata_without_prices.xlsx")

# Split data into features - prices
X_data = train_data[features_X]
y_data = train_data[features_y]

# Define default number of fold
N = 5

# Define the default random_state
# We define it here for convenience
RS = 24

## Handle With Categorical Data and Do Feature Scaling

In [2]:
# We use onehotencoder to do that
from sklearn.preprocessing import OneHotEncoder

# Find the columns to deal with
object_cols = [col for col in X_data.columns if X_data[col].dtype == "object"]

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

# We apply it both X data on train and test (without prices)
OH_cols = pd.DataFrame(OH_encoder.fit_transform(X_data[object_cols]))
OH_X_without_prices_cols = pd.DataFrame(OH_encoder.fit_transform(X_data_without_prices[object_cols]))


# Give them column names (for example "Shift_Auto, Shift_Manual") instead of just integer column names (like "0, 1, 2")
OH_cols.columns = OH_encoder.get_feature_names_out(object_cols)
OH_X_without_prices_cols.columns = OH_encoder.get_feature_names_out(object_cols)

# One-hot encoding removed index; put it back
OH_cols.index = X_data.index
OH_X_without_prices_cols.index = X_data_without_prices.index

# Remove categorical columns (will replace with one-hot encoding)
num_X = X_data.drop(object_cols, axis=1)
num_X_without_prices = X_data_without_prices.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
# From now, We will use OH_X_data instead of X_data
OH_X_data = pd.concat([num_X, OH_cols], axis=1)
OH_X_data_without_prices = pd.concat([num_X_without_prices, OH_X_without_prices_cols], axis=1)

# We will make feature scaling here because when We want my data scaled, We can just make train_test_split with scaled data
# Import scaler
from sklearn.preprocessing import MinMaxScaler

# Set scaler
scaler = MinMaxScaler()

# Scale X Data
OH_X_data_scaled = scaler.fit_transform(OH_X_data) 

# Returns an array, make it df again
OH_X_data_scaled = pd.DataFrame(OH_X_data_scaled, columns=OH_X_data.columns)

# Scale X Data without prices
OH_X_data_without_prices_scaled = scaler.fit_transform(OH_X_data_without_prices)

# Returns an array, make it df again
OH_X_data_without_prices_scaled = pd.DataFrame(OH_X_data_without_prices_scaled, columns=OH_X_data_without_prices.columns)

This is the data before onehotencoder

In [3]:
X_data.head()

,Year,Type,Shift,km,Power,Engine,Seller
0,2004,Diesel,Manual,270000,70,1248,Dealer
1,2004,Gasoline,Manual,164000,75,1199,Dealer
2,2008,Diesel,Auto,300000,90,1248,Dealer
3,2017,Gasoline,Auto,8780,90,1398,Dealer
4,2004,Diesel,Manual,200000,70,1248,Owner


Tis is the data after using onehotencoder

In [4]:
OH_X_data.head()

,Year,km,Power,Engine,Type_Diesel,Type_Gasoline,Type_LPG,Shift_Auto,Shift_Manual,Seller_Dealer,Seller_Owner
0,2004,270000,70,1248,1.0,0.0,0.0,0.0,1.0,1.0,0.0
1,2004,164000,75,1199,0.0,1.0,0.0,0.0,1.0,1.0,0.0
2,2008,300000,90,1248,1.0,0.0,0.0,1.0,0.0,1.0,0.0
3,2017,8780,90,1398,0.0,1.0,0.0,1.0,0.0,1.0,0.0
4,2004,200000,70,1248,1.0,0.0,0.0,0.0,1.0,0.0,1.0


And this is the data with scaled values (for faster iterations)

In [5]:
OH_X_data_without_prices.head()

,Year,km,Power,Engine,Type_Diesel,Type_Gasoline,Type_LPG,Shift_Auto,Shift_Manual,Seller_Dealer,Seller_Owner
0,2013,60000,75,1248,1.0,0.0,0.0,0.0,1.0,0.0,1.0
1,2004,108000,80,1229,0.0,1.0,0.0,0.0,1.0,1.0,0.0
2,2005,134500,80,1229,0.0,1.0,0.0,0.0,1.0,0.0,1.0
3,2000,200000,90,1389,0.0,0.0,1.0,0.0,1.0,1.0,0.0
4,2003,168000,51,1601,1.0,0.0,0.0,0.0,1.0,0.0,1.0


Note: In order to get the best model, we used r2 of the test. After trying cv and r2 test, we find that it maximizes both r2 test and cv score

## Lineer Regression

In [6]:
# Import required libraries
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, cross_val_score

# Prepare Data
X_train, X_test, y_train, y_test = train_test_split(OH_X_data, y_data, random_state=RS)

# Set kfold for evaluate best results for CV
kf5 = KFold(n_splits=N, shuffle=True)

# Make the model
linr = LinearRegression().fit(X_train, y_train)

# Evalueate and save scores
linr_scores = {
     "r2_train": linr.score(X_train, y_train),
     "r2_test": linr.score(X_test, y_test),
     "r2_cv": cross_val_score(linr, X_train, y_train, cv=kf5, scoring="r2").mean()
}

# Print the scores
for k, v in linr_scores.items():
     print(k + ": " + str(v))

r2_train: 0.8969478322104724
r2_test: 0.7871000500553734
r2_cv: 0.8953616147087674


## Polynomial Regression

In [7]:
# Import required libraries
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

# Define variables to evaluate best model
polyr_degree = 0
polyr_score = 0

# Set kfold for evaluate best results for CV
kf5 = KFold(n_splits=N, shuffle=True)

# Loop through different variables to get best model
for this_degree in [1, 2, 3, 4, 5]:
     # Make polynomial features with 'this_degree'
     # We used 'OH_X_data_scaled' to evalueate faster (it will take some time)
     poly = PolynomialFeatures(degree=this_degree)
     X_F1_poly = poly.fit_transform(OH_X_data)
     
     # Preapare polynomial data for regression
     X_train, X_test, y_train, y_test = train_test_split(X_F1_poly, y_data, random_state = RS)
     
     # Make the model with data
     polyr = LinearRegression().fit(X_train, y_train)

     # Get the performance score
     this_score = polyr.score(X_test, y_test)
     
     # We make test with cross validation score, if new score is better than old one, just get new ones
     if (this_score > polyr_score):
          polyr_score = this_score
          polyr_degree = this_degree

# Make the final model with the best variables
poly = PolynomialFeatures(degree=polyr_degree)
X_F1_poly = poly.fit_transform(OH_X_data)

# Prepare data
X_train, X_test, y_train, y_test = train_test_split(X_F1_poly, y_data, random_state=RS)

# Make model
polyr = LinearRegression().fit(X_train, y_train)


# Get and save the r2 values of the model
polyr_scores = {
     'optimal_degree': polyr_degree,
     'r2_train': polyr.score(X_train, y_train),
     'r2_test': polyr.score(X_test, y_test),
     'r2_cv': cross_val_score(polyr, X_train, y_train, cv=kf5, scoring="r2").mean()
}

# Print scores and variables for the model
for k, v in polyr_scores.items():
     print(k + ": " + str(v))


optimal_degree: 3
r2_train: 0.932610636192013
r2_test: 0.7995928390891137
r2_cv: 0.9164351483812607


## Ridge Regression

In [8]:
# Import required libraries
from sklearn.linear_model import Ridge

# Preapare data
X_train, X_test, y_train, y_test = train_test_split(OH_X_data, y_data, random_state = RS)

# Define variables to evaluate best model
ridgr_alpha = 0
ridgr_cv = 0

# Set kfold for evaluate best results for CV
kf5 = KFold(n_splits=N, shuffle=True)

# Loop through different variables to get best model
for this_a in [0.001, 0.005, 0.01, 0.05, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 5, 10, 60, 80, 90, 95, 100, 120, 140, 150, 200, 1000]:
     # Make model with 'this_a' alpha
     ridgr = Ridge(alpha=this_a, random_state=RS).fit(X_train, y_train)

     # Calculate the score with the model just made
     this_score = ridgr.score(X_test, y_test)
     
     # if new score is better than old one, just assign new values to old ones
     if (this_score > ridgr_cv):
          ridgr_cv = this_score
          ridgr_alpha = this_a

# Make the final model with the best variables
ridgr = Ridge(alpha=ridgr_alpha, random_state=RS).fit(X_train, y_train)

# Get and save the r2 values of the model
ridgr_scores = {
     'optimal_alpha': ridgr_alpha,
     'r2_train': ridgr.score(X_train, y_train),
     'r2_test': ridgr.score(X_test, y_test),
     'r2_cv': cross_val_score(ridgr, X_train, y_train, cv=kf5, scoring="r2").mean()
}

# Print scores and variables for the model
for k, v in ridgr_scores.items():
     print(k + ": " + str(v))

optimal_alpha: 90
r2_train: 0.8966922302130962
r2_test: 0.7873473305546449
r2_cv: 0.8949347536749139


c:\Users\talha\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=8.79692e-17): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T


## Lasso Regression

In [9]:
# Import required libraries
from sklearn.linear_model import Lasso

# Preapare data
X_train, X_test, y_train, y_test = train_test_split(OH_X_data, y_data, random_state = RS)

# Define variables to evaluate best model
lassr_alpha = 0
lassr_cv = 0

# Set kfold for evaluate best results for CV
kf5 = KFold(n_splits=N, shuffle=True)

# Loop through different variables to get best model
for this_a in [0.01, 0.05, 0.01, 0.02, 0.1, 0.2, 0.25, 0.3, 0.35, 0.4, 0.5, 1, 5, 10, 16, 14, 18, 20, 25, 30, 40, 60]:
     # Make model with 'this_a' alpha
     lassr = Lasso(alpha=this_a, random_state=RS).fit(X_train, y_train)

     # Calculate the score with the model just made
     this_score = lassr.score(X_test, y_test)

     # if new score is better than old one, just assign new values to old ones
     if (this_score > lassr_cv):
          lassr_cv = this_score
          lassr_alpha = this_a

# Make the final model with the best variables
lassr = Lasso(alpha=lassr_alpha, random_state=RS).fit(X_train, y_train)


# Get and save the r2 values of the model
lassr_scores = {
     'optimal_alpha': lassr_alpha,
     'r2_train': lassr.score(X_train, y_train),
     'r2_test': lassr.score(X_test, y_test),
     'r2_cv': cross_val_score(lassr, X_train, y_train, cv=kf5, scoring="r2").mean()
}

# Print scores and variables for the model
for k, v in lassr_scores.items():
     print(k + ": " + str(v))

optimal_alpha: 5
r2_train: 0.8969444050909176
r2_test: 0.7871074761513857
r2_cv: 0.8958227250875839


## Linear SVM

Note: We had very poor r2 scores but cv score is not that bad.

In [10]:
# Import required libraries
from sklearn.svm import LinearSVC

# Prepare data (with scaled data)
# We use .values.ravel() because We get an error that recommend me to do that
X_train, X_test, y_train, y_test = train_test_split(OH_X_data_scaled, y_data.values.ravel(), random_state=RS)

# Define variables to evaluate best model
linsvc_c = 0
linsvc_cv = 0

# Set kfold for evaluate best results for CV
kf5 = KFold(n_splits=N, shuffle=True) 

# Loop through different variables to get best model
for this_c in [0.001, 0.005, 0.06, 0.08, 0.1, 0.12, 0.14, 0.16, 0.2, 0.3, 0.35, 0.4, 0.45, 0.5]:
    
    # Make model with 'this_c'
    linsvc = LinearSVC(C=this_c, random_state=RS).fit(X_train, y_train)

    # Calculate the score with the model just made
    this_cv = linsvc.score(X_test, y_test)

    # if new score is better than old one, just assign new values to old ones
    if (this_cv > linsvc_cv):
        linsvc_cv = this_cv
        linsvc_c = this_c

# Make model with best variable
linsvc = LinearSVC(C=linsvc_c, random_state=RS).fit(X_train, y_train)

# Get and save r2 scores for the model
linsvc_scores = {
    'optimal_c': linsvc_c,
    'r2_train': linsvc.score(X_train, y_train),
    'r2_test': linsvc.score(X_test, y_test),
    'r2_cv': cross_val_score(linsvc, X_train, y_train, cv=kf5, scoring="r2").mean()
}

# Print scores and variables for the model
for k, v in linsvc_scores.items():
     print(k + ": " + str(v))

optimal_c: 0.06
r2_train: 0.05839080459770115
r2_test: 0.028965517241379312
r2_cv: 0.6997544953462164


## Non-Linear SVM

Note: It takes too long to calculate the model and but still we have very poor results. However, even if it calculates the score with r2 test, it maximizes the cv score.

In [11]:
# Import required libraries
from sklearn.svm import SVC

# Prepare data (with scaled data)
# We use .values.ravel() because We get an error that recommend me to do that
X_train, X_test, y_train, y_test = train_test_split(OH_X_data_scaled, y_data.values.ravel(), random_state=RS)

# Define variables to evaluate best model
svc_c = 1
svc_cv = 0

# Set kfold for evaluate best results for CV
kf5 = KFold(n_splits=N, shuffle=True, random_state=RS)

# Loop through different variables to get best model
for this_c in [10, 1000, 3000, 3500, 4000, 4500, 5000]:
    # Make model with 'this_c' alpha
    svc = SVC(C=this_c, random_state=RS).fit(X_train, y_train)

    # Calculate the score with the model just made
    this_score = svc.score(X_test, y_test)

    # if new score is better than old one, just assign new values to old ones
    if (this_score > svc_cv):
        svc_cv = this_score
        svc_c = this_c

# Make model with best variable
svc = SVC(C=svc_c, random_state=RS).fit(X_train, y_train)

# Get and save r2 scores for the model
svc_scores = {
    'optimal_c': svc_c,
    'r2_train': svc.score(X_train, y_train),
    'r2_test': svc.score(X_test, y_test),
    'r2_cv': cross_val_score(svc, X_train, y_train, cv=kf5, scoring='r2').mean()
}

# Print scores and variables for the model
for k, v in svc_scores.items():
     print(k + ": " + str(v))

optimal_c: 4000
r2_train: 0.5314942528735632
r2_test: 0.05103448275862069
r2_cv: 0.8825659118564708


## Create excel file to export scores

In [18]:
# Create excel file includeing scores etc
# Make a dataframe with required columns
table = pd.DataFrame(columns=["R2 Train", "R2 Test", "CV"])

# Add model scores to the table
table.loc["Linear Regression"] = [linr_scores["r2_train"], linr_scores["r2_test"], linr_scores["r2_cv"].mean()]
table.loc["Polynomial Regression (degree: {})".format(polyr_scores['optimal_degree'])] = [
            polyr_scores["r2_train"], polyr_scores["r2_test"], polyr_scores["r2_cv"].mean()]
table.loc["Ridge Regression (alpha: {})".format(ridgr_scores['optimal_alpha'])] = [
            ridgr_scores["r2_train"], ridgr_scores["r2_test"], ridgr_scores["r2_cv"].mean()]
table.loc["Lasso Regression (alpha: {})".format(lassr_scores['optimal_alpha'])] = [
            lassr_scores["r2_train"], lassr_scores["r2_test"], lassr_scores["r2_cv"].mean()]
table.loc["Linear SVM (c: {})".format(linsvc_scores['optimal_c'])] = [
            linsvc_scores["r2_train"], linsvc_scores["r2_test"], linsvc_scores["r2_cv"].mean()]
table.loc["Non Linear Svm (c: {})".format(svc_scores['optimal_c'])] = [
            svc_scores["r2_train"], svc_scores["r2_test"], svc_scores["r2_cv"].mean()]

# We couldnt find the direct way to export .docx file type
# So we first export as excel table, than convert it to .docx
table.to_excel("Table.xlsx")
table

,R2 Train,R2 Test,CV
Linear Regression,0.896948,0.787100,0.895362
Polynomial Regression (degree: 3),0.932611,0.799593,0.916435
Ridge Regression (alpha: 90),0.896692,0.787347,0.894935
Lasso Regression (alpha: 5),0.896944,0.787107,0.895823
Linear SVM (c: 0.06),0.058391,0.028966,0.699754
Non Linear Svm (c: 4000),0.531494,0.051034,0.882566


## Export Results

We select the Polynomial Regression model because both test score and cv score is the best between them

In [20]:
# Select best model and export Results
best_model = polyr

# Make test data ready for poly regression prediction (the same features polynomial regression have)
X_without_prices_poly = poly.fit_transform(OH_X_data_without_prices)

# Make prediction
results = best_model.predict(X_without_prices_poly)

# Export results (prices are floats in Result.txt)
np.savetxt('Results.txt', results, fmt='%f')